In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <B9202094-7D52-318C-99CF-7034B0E9F28D> /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/O

In [ ]:
df = pd.read_csv('bank.csv')
df

In [ ]:
df.previous.unique()

# Understanding the Dataset

Age:- Age of the customer

Job:- Job of the customer

Marital:- Marital status of the customer (Married or unmarried)

Education:- Education level of the customer

Default:- Has ever defaulted in credit

Balance:- Account Balance

Housing:- If the client has housing loan with the bank

Loan:- If the client has personal loan with the bank

Contact:- Mode of communication

Month:- Month when last contacted

Day:- Day when last contacted

Duration:- Duration of last contact in seconds

Campaign:- Number of times the person was contacted during the current campaign

pdays:- number of days since client was last contacted after the previous campaign

previous:- Number of times the person was contacted during the previous campaigns

poutcome:- outcome of the previous campaign

deposit:- does the client have a deposit with the bank

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

Checking the values in each column

In [ ]:
for column in df.columns:
    print(f"Unique values in {column}:")
    print(df[column].unique(),"\n")

In [ ]:
df

In [ ]:
df.deposit.value_counts()

In [ ]:
df.hist(bins=30, figsize=(15, 10))
plt.tight_layout()
plt.show()

In [ ]:
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']

fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(15, 20))
fig.tight_layout(pad=5.0)

for ax, column in zip(axes.flatten(), categorical_columns):
    sns.countplot(data=df, x=column, ax=ax)
    ax.set_title(f'Distribution of {column}')
    ax.set_xlabel(column)
    ax.set_ylabel('Count')

axes[0, 0].tick_params(axis='x', rotation=45)
plt.show()


In [ ]:
numerical_columns = df.select_dtypes(include=['int64']).columns
numerical_columns = numerical_columns.drop('day')

fig, axes = plt.subplots(nrows=2, ncols=len(numerical_columns)//2 + len(numerical_columns)%2, figsize=(20, 15))
fig.tight_layout(pad=5.0)
palette = sns.color_palette("husl", len(numerical_columns))

for ax, column in zip(axes.flatten(), numerical_columns):
    sns.boxplot(data=df, y=column, ax=ax)
    ax.set_title(f'Boxplot of {column}')
    ax.set_ylabel(column)

plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(20, 25))
fig.tight_layout(pad=5.0)

for ax, column in zip(axes.flatten(), categorical_columns):
    sns.boxplot(data=df, x=column, y='balance', ax=ax)
    ax.set_title(f'Boxplot of balance by {column}')
    ax.set_xlabel(column)
    ax.set_ylabel('Balance')
    axes[0, 0].tick_params(axis='x', rotation=45)
plt.show()

In [ ]:
def remove_outliers(df, columns):
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

all_columns = df.select_dtypes(include=['int64', 'float64']).columns  
df1 = remove_outliers(df, all_columns)
df1

In [ ]:
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']

label_encoder = LabelEncoder()
df2 = df1.copy()

for column in categorical_columns:
    df2[column] = label_encoder.fit_transform(df1[column])

df2

In [ ]:
X = df2.drop('deposit', axis=1)
y = df2['deposit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

accuracy_dt = (y_pred == y_test).mean()
accuracy_dt_train = (dt.predict(X_train) == y_train).mean()
print(f"Training Accuracy: {accuracy_dt_train}")
print(f"Test Accuracy: {accuracy_dt}")

In [ ]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

accuracy_rf = (y_pred == y_test).mean()
accuracy_rf_train = (rf.predict(X_train) == y_train).mean()
print(f"Training Accuracy: {accuracy_rf_train}")
print(f"Test Accuracy: {accuracy_rf}")

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

rf_random.fit(X_train, y_train)

best_params = rf_random.best_params_
best_params

In [ ]:
best_params

In [ ]:
best_rf = RandomForestClassifier(**best_params, random_state=42)

best_rf.fit(X_train, y_train)

y_pred_best_rf = best_rf.predict(X_test)
accuracy_best_rf_train = (best_rf.predict(X_train) == y_train).mean()
print(f"Training Accuracy: {accuracy_best_rf_train}")
print(f"Test Accuracy: {accuracy_best_rf}")

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)

gb.fit(X_train, y_train)

y_pred = gb.predict(X_test)

accuracy_gb = (y_pred == y_test).mean()
accuracy_gb

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

gb_grid = GridSearchCV(estimator=gb, param_grid=param_grid_gb, cv=3, n_jobs=-1, verbose=2)

gb_grid.fit(X_train, y_train)

best_params_gb = gb_grid.best_params_
best_params_gb

In [ ]:
best_gb = GradientBoostingClassifier(**best_params_gb, random_state=42)

best_gb.fit(X_train, y_train)

y_pred_best_gb = best_gb.predict(X_test)

accuracy_best_gb_train = (best_gb.predict(X_train) == y_train).mean()
accuracy_best_gb_test = (y_pred_best_gb == y_test).mean()
print(f"Training Accuracy: {accuracy_best_gb_train}")
print(f"Test Accuracy: {accuracy_best_gb_test}")

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm = SVC(random_state=42)

svm.fit(X_train, y_train)

y_pred_svm = svm.predict(X_test)

accuracy_svm_train = accuracy_score(y_train, svm.predict(X_train))
print(f"Training Accuracy: {accuracy_svm_train}")
print(f"Test Accuracy: {accuracy_svm}")

In [ ]:
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)

accuracy_xgb_train = (xgb.predict(X_train) == y_train).mean()
accuracy_xgb_test = (y_pred_xgb == y_test).mean()

print(f"Training Accuracy: {accuracy_xgb_train}")
print(f"Test Accuracy: {accuracy_xgb_test}")

In [ ]:
# Model names and their corresponding accuracies
models = ['Decision Tree', 'Random Forest', 'Best Random Forest', 'Gradient Boosting', 'Best Gradient Boosting', 'SVM']
accuracies = [accuracy_dt, accuracy_rf, accuracy_best_rf, accuracy_gb, accuracy_best_gb_test, accuracy_svm]

plt.figure(figsize=(16, 6))
plt.bar(models, accuracies, color=['blue', 'green', 'red', 'purple', 'orange', 'cyan'])

for i, v in enumerate(accuracies):
    plt.text(i, v + 0.01, f"{v:.3f}", ha='center')
    
plt.xlabel('Models')
plt.xticks(rotation=45)
plt.ylabel('Accuracy')
plt.title('Model Accuracies')
plt.ylim(0, 1)
plt.show()